# Notebook for analysis of Magnum data by concatenating analysis datasets together
The analysis datasets are stored in the netcdf files created by the script magnumrun_postrun_initial_analysis.py and kept in backup on Freia's funscratch directory

### Imports:

In [1]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import xarray as xr 
import scipy.stats as stat
import sys
import os
import glob
import re
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.core.ivdata as iv
import flopter.core.lputils as lp
import flopter.magnum.database as ut
import flopter.core.fitters as fts

In [2]:
import importlib
importlib.reload(xr)

<module 'xarray' from '/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/__init__.py'>

## Construction of the metadata dataframe
Metadata about the analysed xarray datasets is compiled into a pandas dataframe for simple manipulation

In [1]:
# Create analysed dataset metadata 

path_to_datasets = '/home/jleland/Data/external/magnum/'
# path_to_analysed_datasets = 'analysed_2'
# path_to_analysed_datasets = 'analysed_3'
# path_to_analysed_datasets = 'analysed_4'
# path_to_analysed_datasets = 'test'
path_to_analysed_datasets = 'analysed_3_downsampled'
os.chdir(path_to_datasets)

NameError: name 'os' is not defined

In [9]:
analysed_dataset_fns = glob.glob(path_to_analysed_datasets + '/*.nc')
analysed_dataset_fns.sort()

analysed_infos = []

for i, anal_ds in enumerate(analysed_dataset_fns):
    match = re.search("\/a{1}([0-9]{3})_([0-9]{3})_([0-9]{19})\.nc", anal_ds)
    shot_index = int(match.group(1))
    adc_index = int(match.group(2))
    shot_timestamp = int(match.group(3))
    shot_time = ut.human_datetime_str(int(match.group(3)))
    
    ds = xr.open_dataset(anal_ds)
    time_len = len(ds['time'])
    sweep_len = len(ds['sweep'])
    
    analysed_infos.append({
        'adc_index':adc_index, 
        'shot_number':shot_index, 
        'shot_timestamp':shot_timestamp, 
        'shot_time':shot_time, 
        'filename':anal_ds,
        'time_len': time_len,
        'sweep_len': sweep_len
    })

analysed_infos_df = pd.DataFrame(analysed_infos).set_index('adc_index')

In [10]:
analysed_infos_df.groupby('time_len').count()

,shot_number,shot_timestamp,shot_time,filename,sweep_len
time_len,,,,,
5000,80,80,80,80,80


In [11]:
analysed_infos_df.groupby('sweep_len').count().plot()

---
## Saving / loading section
The metadata doesn't need to be reconstructed _every_ time, so we can store it in a .csv file output by pandas. The following 2 methods save and load, use whichever is appropriate

In [12]:
metadata_filename = f'{path_to_analysed_datasets}_xr_metadata.csv'

In [4]:
# Loading the metadata .csv file into a pandas dataframe
analysed_infos_df = pd.read_csv(metadata_filename).set_index('adc_index')

In [13]:
# Saving the metadata.csv file from the constructed pandas dataframe
analysed_infos_df.to_csv(metadata_filename)

End of save/load segment

---

## Index choice section

In [5]:
magnum_probes = lp.MagnumProbes()

In [14]:
indices = analysed_infos_df.where(analysed_infos_df.time_len >= 500).dropna().index.values
indices
print(len(indices), len(analysed_infos_df))

80 80


In [6]:
# list(analysed_infos_df.groupby('time_len'))

In [8]:
# Axial Scan
indices = [132,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189]

In [7]:
# 100A, 5slm (He), 0.8T
indices = [357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386]

In [24]:
# 100A, 5slm (He), 1.2T
indices = [245, 246, 247, 248, 249, 250, 251, 252, 254, 255, 256, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271]

In [7]:
# 110A, 5slm (H), 1.2T
indices = [320, 321, 322, 323, 324, 325, 327, 328, 329, 330, 331, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346]

In [6]:
indices = [357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386]\
          + [132,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189]\
          + [320, 321, 322, 323, 324, 325, 327, 328, 329, 330, 331, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346]

In [7]:
analysed_infos_df.loc[indices]

,filename,shot_number,shot_time,shot_timestamp,sweep_len,time_len
adc_index,,,,,,
357,analysed_3/a361_357_6699321692420543488.nc,361,2019-06-06 08:49:41,6699321692420543488,223,5000
358,analysed_3/a362_358_6699322895963984896.nc,362,2019-06-06 08:54:21,6699322895963984896,223,5000
359,analysed_3/a363_359_6699323408786381824.nc,363,2019-06-06 08:56:21,6699323408786381824,219,5000
360,analysed_3/a364_360_6699323711970046976.nc,364,2019-06-06 08:57:31,6699323711970046976,215,5000
361,analysed_3/a365_361_6699324107152520192.nc,365,2019-06-06 08:59:03,6699324107152520192,223,5000
362,analysed_3/a366_362_6699324505470350336.nc,366,2019-06-06 09:00:36,6699324505470350336,222,5000
363,analysed_3/a367_363_6699324755567907840.nc,367,2019-06-06 09:01:34,6699324755567907840,216,5000
364,analysed_3/a368_364_6699324990246430720.nc,368,2019-06-06 09:02:29,6699324990246430720,220,5000
365,analysed_3/a369_365_6699325234288040960.nc,369,2019-06-06 09:03:26,6699325234288040960,225,5000


In [6]:
# for i in analysed_infos_df.loc[indices].index:
#     row = analysed_infos_df.loc[i]
#     print(row)

## Load adc file metadata

In [15]:
os.chdir('/home/jleland/Data/external/magnum/')
meta_data_ds = xr.open_dataset('all_meta_data.nc')
print(meta_data_ds)

<xarray.Dataset>
Dimensions:                 (shot_number: 523, ts_radial_pos: 46)
Coordinates:
  * shot_number             (shot_number) int32 0 1 2 3 4 ... 519 520 521 522
    ts_number               (shot_number) float64 ...
    ts_timestamp            (shot_number) float64 ...
    ts_time                 (shot_number) datetime64[ns] ...
    adc_index               (shot_number) float64 ...
    adc_time                (shot_number) datetime64[ns] ...
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
Data variables:
    adc_filename            (shot_number) object ...
    ts_density              (shot_number, ts_radial_pos) float64 ...
    ts_temperature          (shot_number, ts_radial_pos) float64 ...
    ts_d_density            (shot_number, ts_radial_pos) float64 ...
    ts_d_temperature        (shot_number, ts_radial_pos) float64 ...
    adc_folder              (shot_number) object ...
    adc_calibration_index   (shot_number) object ...
    adc_4_

# Analysis of shot / adc_file index selection

In [16]:
def preprocess_sel(ds, sweep=slice(0,997)):
    return ds.sel(sweep=sweep)

def preprocess_average(ds):
    ds = ds.reset_index('time', drop=True)
    ds_avg = ds.mean('sweep')
    ds_avg = ds_avg.assign({'d_current': ds.std('sweep')['current']})
    return ds_avg

def preprocess_average_downsample(ds, downsample_to=500):
    ds = ds.reset_index('time', drop=True)
    dsf = int(len(ds.time) / downsample_to)
    ds = ds.sel(time=slice(0, len(ds.time), dsf))
    ds_avg = ds.mean('sweep')
    ds_avg = ds_avg.assign({'d_current': ds.std('sweep')['current']})
    return ds_avg

In [17]:
analysed_metadata_oi = analysed_infos_df.loc[indices]

files_oi = analysed_metadata_oi['filename'].values
shot_numbers = analysed_metadata_oi['shot_number'].values
shot_numbers_da = xr.DataArray(shot_numbers, dims=['shot_number'], name='shot_number')
min_sweep_len = analysed_metadata_oi['sweep_len'].min()
print(min_sweep_len)

47


## Attempt at concatenating datasets manually

In [11]:
# ds_list = [xr.open_dataset(fn) for fn in files_oi]

In [16]:
indices_to_pop = []
for i, ds in enumerate(ds_list):
    if len(ds.time) < 500:
        indices_to_pop.append(i)
        continue
#     ds = preprocess_average_downsample(ds)
#     ds = ds.sel(time=slice())
#     ds = preprocess_average(ds)

In [17]:
print(indices_to_pop)

[]


In [11]:
# combined_ds = xr.concat(ds_list, shot_numbers_da, fill_value=np.nan)

# Analysis of Helium shot @120A, 5slm He, 0.8T

In [19]:
os.chdir('/home/jleland/Data')

In [10]:
os.chdir('/home/jleland/Data/external/magnum')

In [12]:
helium_meta_ds = meta_data_ds.sel(shot_number=shot_numbers)
helium_meta_ds.max('ts_radial_pos')

/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:223: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)


<xarray.Dataset>
Dimensions:                 (shot_number: 87)
Coordinates:
  * shot_number             (shot_number) int32 361 362 363 364 ... 351 352 353
    ts_number               (shot_number) float64 ...
    ts_timestamp            (shot_number) float64 ...
    ts_time                 (shot_number) datetime64[ns] ...
    adc_index               (shot_number) float64 ...
    adc_time                (shot_number) datetime64[ns] ...
Data variables:
    adc_filename            (shot_number) object '2019-06-06 09h 49m 51s TT_06699321692420543855.adc' ... '2019-06-05 17h 03m 59s TT_06699062430372281696.adc'
    ts_density              (shot_number) float64 1.037e+20 ... 1.896e+19
    ts_temperature          (shot_number) float64 7.225 8.182 ... 5.021 4.254
    ts_d_density            (shot_number) float64 0.0 0.0 ... 8.427e+16
    ts_d_temperature        (shot_number) float64 0.0 0.0 0.0 ... 0.03036 0.0308
    adc_folder              (shot_number) object '2019-06-06_Leland/' ... '2019-

In [13]:
tilt_da = helium_meta_ds[['shot_tilt', 'adc_4_probe', 'adc_5_probe']]
tilt_da['shot_tilt'] = tilt_da.shot_tilt.round()
tilt_da


<xarray.Dataset>
Dimensions:       (shot_number: 87)
Coordinates:
    ts_time       (shot_number) datetime64[ns] ...
    adc_index     (shot_number) float64 ...
  * shot_number   (shot_number) int32 361 362 363 364 365 ... 350 351 352 353
    ts_number     (shot_number) float64 ...
    adc_time      (shot_number) datetime64[ns] ...
    ts_timestamp  (shot_number) float64 ...
Data variables:
    shot_tilt     (shot_number) float64 10.0 9.0 8.0 7.0 ... 7.0 8.0 9.0 10.0
    adc_4_probe   (shot_number) object 'S' 'S' 'S' 'S' 'S' ... 'R' 'R' 'R' 'R'
    adc_5_probe   (shot_number) object 'L' 'L' 'L' 'L' 'L' ... 'B' 'B' 'B' 'B'

In [16]:
combined_ds = xr.open_mfdataset(files_oi, concat_dim=shot_numbers_da, preprocess=preprocess_average_downsample, parallel=True, combine='nested')
# combined_ds = combined_ds.load()
combined_ds

/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: In xarray version 0.14 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  """Entry point for launching an IPython kernel.


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [15]:
combined_all_ds = xr.merge([combined_ds, helium_meta_ds.rename({'shot_time':'sweep_time'})])
combined_all_ds

<xarray.Dataset>
Dimensions:                 (direction: 2, probe: 4, shot_number: 440, time: 500, ts_radial_pos: 46)
Coordinates:
  * probe                   (probe) object 'B' 'L' 'R' 'S'
  * direction               (direction) object 'up' 'down'
  * shot_number             (shot_number) int64 3 4 5 6 7 ... 478 479 480 481
    ts_number               (shot_number) float64 ...
    ts_timestamp            (shot_number) float64 ...
    ts_time                 (shot_number) datetime64[ns] ...
    adc_index               (shot_number) float64 ...
    adc_time                (shot_number) datetime64[ns] ...
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
Dimensions without coordinates: time
Data variables:
    voltage                 (shot_number, probe, direction, time) float64 dask.array<shape=(440, 4, 2, 500), chunksize=(1, 4, 2, 500)>
    current                 (shot_number, probe, direction, time) float64 dask.array<shape=(440, 4, 2, 500), chunksize=(

In [16]:
combined_all_ds.to_netcdf('all_combined_ds.nc')

In [22]:
anglescan_ds = xr.merge([combined_ds, helium_meta_ds.rename({'shot_time':'sweep_time'})]).assign_coords(tilt=helium_meta_ds['shot_tilt'].round())
anglescan_ds = anglescan_ds.swap_dims({'shot_number':'tilt'}) #.mean('direction')

# Reorganise to make tilt and probe dimensions
probes_1 = anglescan_ds.sel(probe=['S', 'L']).where(np.isfinite(anglescan_ds.voltage), drop=True) 
probes_2 = anglescan_ds.sel(probe=['R', 'B']).where(np.isfinite(anglescan_ds.voltage), drop=True)

anglescan_ds = xr.concat([probes_1, probes_2], dim='probe').mean('direction')
anglescan_ds

KeyboardInterrupt: 

In [18]:
colourmap = plt.get_cmap('nipy_spectral')
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=[colourmap(k) for k in np.linspace(0, 1, 12)])

iv_fig, iv_ax = plt.subplots(2, 2, sharex=True)
for i, probe in enumerate(anglescan_ds['probe'].values):
    iv_ax[i%2][i//2].set_title(probe)
    
    ds = anglescan_ds.sel(probe=probe)
    ds.set_coords('voltage')['current'].plot.line(ax=iv_ax[i%2][i//2], x='voltage', hue='tilt')
    iv_ax[i%2][i//2].set_ylabel('I (A)')
    iv_ax[i%2][i//2].set_xlabel(r'$V_P$ (V)')
    
# plt.tight_layout()
plt.show()

In [23]:
ts_temp = anglescan_ds.ts_temperature.mean(['probe', 'tilt', 'time'])
ts_d_temp = anglescan_ds.ts_temperature.std(['probe', 'tilt', 'time'])
ts_dens = anglescan_ds.ts_density.mean(['probe', 'tilt', 'time'])
ts_d_dens = anglescan_ds.ts_density.std(['probe', 'tilt', 'time'])

print(ts_temp.shape)

fig, ax = plt.subplots(2)
ax[0].set_title('Temperature')
ax[1].set_title('Density')
# ax[0].errorbar(helium_anglescan_ds.ts_radial_pos, ts_temp, yerr=helium_anglescan_ds.ts_d_temperature.mean('shot_number'), color='silver', ecolor='silver')
ax[0].errorbar(anglescan_ds.ts_radial_pos, ts_temp, yerr=ts_d_temp)
ax[1].errorbar(anglescan_ds.ts_radial_pos, ts_dens, yerr=ts_d_dens)
# ts_temp.plot.line(ax=ax[0])
# ts_dens.plot.line(ax=ax[1])
for i in [0, 1]:
    for probe_pos in [6, -4, -14, -24]:
        ax[i].axvline(x=probe_pos, color='black', linewidth=1, linestyle='dashed')
plt.show()


/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:159: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1628: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


(46,)


In [64]:
# helium_anglescan_ds.where(helium_anglescan_ds.ts_temperature == helium_anglescan_ds.max('ts_radial_pos')['ts_temperature'])
middle_temps = anglescan_ds.isel(ts_radial_pos=anglescan_ds['ts_temperature'].argmax('ts_radial_pos'))['ts_radial_pos']
middle_dens = anglescan_ds.isel(ts_radial_pos=anglescan_ds['ts_density'].argmax('ts_radial_pos'))['ts_radial_pos']
middle_combo = np.concatenate([middle_temps.values, middle_dens.values])

print(middle_dens.mean().values, middle_dens.std().values / 22)
print(middle_temps.mean().values, middle_temps.std().values / 22)
print(middle_combo.mean(), middle_combo.std() / 22)

1.0190559463067488 0.22890503939904097
-2.4791958277875725 0.03542135850450465
-0.730069940740412 0.18206387533790472


In [65]:
probe_temps = {}
probe_denss = {}
for probe, probe_pos in {'L':6, 'S':-4, 'B':-14, 'R':-24}.items():
    probe_temps[probe] = ts_temp.interp(ts_radial_pos=probe_pos).values
    probe_denss[probe] = ts_dens.interp(ts_radial_pos=probe_pos).values
    
print(probe_temps)
print(probe_denss)

{'L': array(1.42282395), 'S': array(3.61245318), 'B': array(0.65816987), 'R': array(0.18098454)}
{'L': array(1.54466051e+19), 'S': array(1.59204721e+19), 'B': array(9.22679553e+18), 'R': array(2.65937481e+18)}


In [29]:
# helium_anglescan_ds

In [66]:
# fit and plot for a single probe
probes = ['S', 'L', 'B', 'R']
dens = {label: [] for label in probes}
temps = {label: [] for label in probes}
fig, axes = plt.subplots(3)

probe = 'S'
for tilt in tilt_da.values:
    ds = helium_anglescan_ds.sel(probe=probe, direction='up').max('ts_radial_pos')
    ds = ds.where(np.logical_and(np.isfinite(ds.voltage), ds.tilt == tilt), drop=True)
#     print(ds['current'].values[0])

#     print(shot_iv.keys(), shot_iv['sigma'])
    try:
        shot_iv = iv.IVData(ds['voltage'].values[0],
                    -ds['current'].values[0],
                    ds['shot_time'].values[0],
                    estimate_error_fl=True)
#         iv_fitter = fts.FullIVFitter()
#         shot_fit = iv_fitter.fit_iv_data(shot_iv, sigma=shot_iv['sigma'])
        shot_fit = shot_iv.multi_fit(sat_region=-60)
        dens[probe].append(magnum_probes[probe].get_density(shot_fit.get_isat(), shot_fit.get_temp(), alpha=np.radians(tilt)))
        temps[probe].append(shot_fit.get_temp())
        axes[2].plot(*shot_fit.get_fit_plottables(), label=f'tilt = {tilt}')
        axes[2].plot(*shot_fit.get_raw_plottables(), color='silver')
    except:
        dens[probe].append(np.nan)
        temps[probe].append(np.nan)

axes[0].plot(tilt_da.values, dens[probe], 'x', label=probe)
axes[0].axhline(y=probe_denss[probe], color='red', linewidth=0.5, linestyle='dashed')

axes[1].plot(tilt_da.values, temps[probe], 'x', label=probe)
axes[1].axhline(y=probe_temps[probe], color='red', linewidth=0.5, linestyle='dashed')

axes[2].legend()

TypeError: 'method' object is not iterable

In [62]:
# fit and plot for all probes
probes = ['S', 'L', 'B', 'R']
denss = {label: [] for label in probes}
temps = {label: [] for label in probes}
fits = {label: [] for label in probes}

fig, axes = plt.subplots(4, 2)
colours = ['blue', 'orange', 'green', 'red']

for i, probe in enumerate(probes):
    for tilt in tilt_da.values:
        ds = helium_anglescan_ds.sel(probe=probe).mean('direction').max('ts_radial_pos')
        ds = ds.where(np.logical_and(np.isfinite(ds.voltage), ds.tilt == tilt), drop=True)
        
        try:
            shot_iv = iv.IVData(ds['voltage'].values[0],
                            -ds['current'].values[0],
                            ds['shot_time'].values[0],
                            estimate_error_fl=True)
#             iv_fitter = fts.FullIVFitter()
#             shot_fit = iv_fitter.fit_iv_data(shot_iv, sigma=shot_iv['sigma'])
            shot_fit = shot_iv.multi_fit(sat_region=-60)
            fits[probe].append(shot_fit)
            denss[probe].append(magnum_probes[probe].get_density(shot_fit.get_isat(), shot_fit.get_temp(), alpha=np.radians(tilt)))
            temps[probe].append(shot_fit.get_temp())
        except :
            denss[probe].append(np.nan)
            temps[probe].append(np.nan)
    
    axes[i][0].set_title(r'$n_e$')
    axes[i][0].set_xlabel(r'$\theta_{\perp}$ / degrees')
    axes[i][0].set_ylabel(r'$n_e$ / m$^{-3}$')
    axes[i][0].plot(tilt_da.values, denss[probe], '+', label=probe, color=colours[i])
    axes[i][0].axhline(y=probe_denss[probe], color=colours[i], linewidth=0.5, linestyle='dashed')
    axes[i][0].legend()

    axes[i][1].set_title(r'$T_e$')
    axes[i][1].set_xlabel(r'$\theta_{\perp}$ / degrees')
    axes[i][1].set_ylabel(r'$T_e$ / eV')
    axes[i][1].plot(tilt_da.values, temps[probe], '+', label=probe, color=colours[i])
    axes[i][1].axhline(y=probe_temps[probe], color=colours[i], linewidth=0.5, linestyle='dashed')
    axes[i][1].legend()

# plt.tight_layout()
plt.show()

/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:159: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:220: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwds)
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:159: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:220: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwds)
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:159: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/jleland/Coding/Environments/python3.6/li

In [34]:
fig, axes = plt.subplots(2, 2)
base_fmt_str = r'$\theta$ = {:.2g}, $T_e$={:.2g}, $n_e$={:.2g}'

for i, probe in enumerate(probes[0:3]):
    probe_axes = axes[i//2][i%2]
    for j, fit in enumerate(fits[probe]):
        probe_axes.plot(*fit.get_raw_plottables(), color='silver')
        label_str = base_fmt_str.format(tilt_da.values[j], temps[probe][j], denss[probe][j])
        probe_axes.plot(*fit.get_fit_plottables())
    
    anal_voltage = np.linspace(-100, -20, 100)
    
    for tilt in [1.0,10.0]:
        v_f = -(1/tilt) * 10 * probe_temps[probe]
        anal_current = magnum_probes[probe].get_analytical_iv(anal_voltage, v_f, np.radians(tilt), probe_temps[probe], probe_denss[probe], print_fl=True)
#         print(v_f, tilt, probe_temps[probe], probe_denss[probe])
#         print(anal_current)
        label_str = base_fmt_str.format(tilt, probe_temps[probe], probe_denss[probe])
        probe_axes.plot(anal_voltage, anal_current, label='TS IV ({})'.format(label_str), color='black', linestyle='dashed')
    
    probe_axes.legend()
    probe_axes.set_title(f'Probe {probe}')
    probe_axes.set_ylim(bottom=-0.1)
plt.show()

TypeError: unsupported format string passed to numpy.ndarray.__format__

In [35]:
tilt = 10.0
i = 10
probe = 'S'
print(i, tilt)


positions = np.array([-7, 6])

fig, axes = plt.subplots()
ts_temp.plot()

# axes.axvline(x=positions[0], color='red')
# axes.vlines([positions[0] + (i*10) for i in np.arange(-1, 3)], -0.5, 7.5, color='red')

# axes.axvline(x=positions[1], color='blue')
# axes.vlines([positions[1] + (i*10) for i in np.arange(-1, 3)], -0.5, 7.5, color='blue')

positions = np.linspace(-8, -6, 10)
fig, axes = plt.subplots()

fit = fits[probe][i]
axes.plot(*fit.get_raw_plottables(), color='silver')
label_str = base_fmt_str.format(tilt_da.values[j], temps[probe][i], denss[probe][i])
axes.plot(*fit.get_fit_plottables())

for pos in positions:
    temp = ts_temp.interp(ts_radial_pos=pos).values
    dens = ts_dens.interp(ts_radial_pos=pos).values
    v_f = temp * -10
#     anal_voltage = np.linspace(-100, -20, 100)
    anal_voltage = fit.raw_x
    anal_current = magnum_probes[probe].get_analytical_iv(anal_voltage, v_f, np.radians(tilt), temp, dens, print_fl=True)
    label_str = base_fmt_str.format(tilt, temp, dens)
    axes.plot(anal_voltage, anal_current, label=f'pos = {pos}', linestyle='dashed')

axes.legend()
axes.set_ylim(bottom=0.0)

10 10.0


IndexError: list index out of range

# Non-sweep Averaging

In [36]:
combined_ds = xr.open_mfdataset(files_oi, concat_dim=shot_numbers_da, preprocess=preprocess_sel)
combined_ds = combined_ds.load()
combined_ds

<xarray.Dataset>
Dimensions:      (direction: 2, probe: 4, shot_number: 22, sweep: 997, time: 150)
Coordinates:
  * probe        (probe) object 'B' 'L' 'R' 'S'
  * time         (time) float64 0.0 0.0001 0.0001 ... 0.0049 0.0049 0.0049
  * direction    (direction) object 'up' 'down'
  * shot_number  (shot_number) int64 329 330 331 332 333 ... 349 350 351 352 353
Dimensions without coordinates: sweep
Data variables:
    voltage      (shot_number, probe, direction, sweep, time) float64 nan ... nan
    current      (shot_number, probe, direction, sweep, time) float64 nan ... nan
    shot_time    (shot_number, probe, direction, sweep, time) float64 nan ... nan
    start_time   (shot_number, probe, direction, sweep) float64 nan nan ... nan

In [37]:
hydrogen_anglescan_ds = xr.merge([combined_ds, helium_meta_ds.rename({'shot_time':'sweep_time'})]).assign_coords(tilt=helium_meta_ds['shot_tilt'].round())
hydrogen_anglescan_ds

<xarray.Dataset>
Dimensions:                 (direction: 2, probe: 4, shot_number: 22, sweep: 997, time: 150, ts_radial_pos: 46)
Coordinates:
  * probe                   (probe) object 'B' 'L' 'R' 'S'
  * time                    (time) float64 0.0 0.0001 0.0001 ... 0.0049 0.0049
  * direction               (direction) object 'up' 'down'
  * shot_number             (shot_number) int32 329 330 331 332 ... 351 352 353
    ts_number               (shot_number) float64 ...
    ts_timestamp            (shot_number) float64 ...
    ts_time                 (shot_number) datetime64[ns] ...
    adc_index               (shot_number) float64 ...
    adc_time                (shot_number) datetime64[ns] ...
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
    tilt                    (shot_number) float64 1.0 2.0 3.0 ... 8.0 9.0 10.0
Dimensions without coordinates: sweep
Data variables:
    voltage                 (shot_number, probe, direction, sweep, time) float64 na

In [38]:
probe = 'R'

# shot_number = 343
shot_number = 345

In [39]:
weird_ds = hydrogen_anglescan_ds.sel(probe=probe, shot_number=shot_number).max('ts_radial_pos')
# print(weird_ds)
# weird_ds = weird_ds.where(np.isfinite(weird_ds.voltage), drop=True)
print(weird_ds)

<xarray.Dataset>
Dimensions:                 (direction: 2, sweep: 997, time: 150)
Coordinates:
    probe                   <U1 'R'
  * time                    (time) float64 0.0 0.0001 0.0001 ... 0.0049 0.0049
  * direction               (direction) object 'up' 'down'
    shot_number             int32 345
    ts_number               float64 ...
    ts_timestamp            float64 ...
    ts_time                 datetime64[ns] ...
    adc_index               float64 ...
    adc_time                datetime64[ns] ...
    tilt                    float64 2.0
Dimensions without coordinates: sweep
Data variables:
    voltage                 (direction, sweep, time) float64 2.365 nan ... nan
    current                 (direction, sweep, time) float64 0.0001562 ... nan
    shot_time               (direction, sweep, time) float64 1.015 nan ... nan
    start_time              (direction, sweep) float64 1.015 1.025 ... 10.98
    adc_filename            object '2019-06-05 16h 47m 40s TT_06699058

/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nputils.py:220: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwds)


In [45]:
plt.figure()
for sweep in weird_ds['sweep'].values[::10]:
    sweep_ds = weird_ds.sel(sweep=sweep)
#     plt.plot(sweep_ds['time'].values, sweep_ds['voltage'].values)
    plt.plot(sweep_ds['voltage'].values, sweep_ds['current'].values)
    
sweep_avg = weird_ds.mean('sweep')
plt.plot(sweep_avg['voltage'].values, sweep_avg['current'].values, 'k', label='Average')
plt.show()

/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:159: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [44]:
zeroed_ds = weird_ds - sweep_avg
bins = np.linspace(-0.05,0.05,100)

plt.figure()
for time in zeroed_ds['time'].values[0:1]:
    time_ds = zeroed_ds.sel(time=time, direction='up')
    hist = time_ds['current'].plot.hist(bins=bins)
    time_ds = zeroed_ds.sel(time=time, direction='down')
    hist = time_ds['current'].plot.hist(bins=bins)
    
# sweep_avg = weird_ds.mean('sweep')
# plt.plot(sweep_avg['time'].values, sweep_avg['voltage'].values, 'b', label='Average')
plt.show()

# Trying to find problem with time dimension
Found it by looking at teh individual time values on each dataset and noticing slight differences. Fixed by dropping the coordinate from time, which can be added back in later if necessary.

In [9]:
analysed_infos_df['shot_index'].loc[indices[0]]

143

In [22]:
ds_list = []
slicey = slice(0, 2)
for i, ds_file in enumerate(files_oi[slicey]):
    ds = xr.open_dataset(ds_file).sel(sweep=slice(0,699)).expand_dims({'shot_number':[analysed_infos_df['shot_index'].loc[indices[i]]]}).reset_index('time').mean('direction').mean('sweep')
    ds_list.append(ds)

shot_number_da = xr.DataArray(analysed_infos_df.loc[indices[slicey]]['shot_index'].values, dims=['shot_number'], name='shot_number')
autocombined_ds = xr.concat(ds_list, dim=shot_number_da)
print(autocombined_ds)

<xarray.Dataset>
Dimensions:      (probe: 2, shot_number: 2, time: 5000)
Coordinates:
  * probe        (probe) object 'S' 'L'
  * shot_number  (shot_number) int64 143 144
    time_        (shot_number, time) float64 0.0 1e-06 ... 0.004998 0.004999
Dimensions without coordinates: time
Data variables:
    voltage      (shot_number, probe, time) float64 88.51 88.59 ... -967.3
    current      (shot_number, probe, time) float64 0.2893 0.2893 ... -0.01502
    shot_time    (shot_number, probe, time) float64 4.496 4.496 ... 4.496 4.496
    start_time   (shot_number, probe) float64 4.494 4.494 4.494 4.494


In [24]:
for ds_file in files_oi[0:1]:
    ds = xr.open_dataset(ds_file)
    print(ds['time'].max())
    print(ds_file, ds['time'].values, ds.mean('direction').mean('sweep')['time'].values)

<xarray.DataArray 'time' ()>
array(0.004999)
analysed_1/a143_148_6698301520675232768.nc [0.0000000e+00 1.0000001e-06 2.0000002e-06 ... 4.9970005e-03 4.9980005e-03
 4.9990005e-03] [0.0000000e+00 1.0000001e-06 2.0000002e-06 ... 4.9970005e-03 4.9980005e-03
 4.9990005e-03]
